In [ ]:
import heapq
from copy import deepcopy
from IPython.display import display, HTML

class EstadoPuzzle:
    def __init__(self, tablero, fila_vacia, columna_vacia, nivel=0, estado_anterior=None):
        self.tablero = deepcopy(tablero)
        self.fila_vacia = fila_vacia
        self.columna_vacia = columna_vacia
        self.nivel = nivel
        self.estado_anterior = estado_anterior
        self.costo_total = self.calcular_costo_total()
    
    def calcular_costo_total(self):
        """Calcula el costo total usando distancia Manhattan"""
        costo_heuristico = 0
        for i in range(3):
            for j in range(3):
                if self.tablero[i][j] != 0:
                    fila_objetivo = (self.tablero[i][j] - 1) // 3
                    columna_objetivo = (self.tablero[i][j] - 1) % 3
                    costo_heuristico += abs(i - fila_objetivo) + abs(j - columna_objetivo)
        return costo_heuristico + self.nivel
    
    def es_estado_final(self):
        estado_final = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
        return self.tablero == estado_final
    
    def generar_hijos(self):
        movimientos = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Arriba, abajo, izquierda, derecha
        hijos = []
        
        for df, dc in movimientos:
            nueva_fila = self.fila_vacia + df
            nueva_col = self.columna_vacia + dc
            
            if 0 <= nueva_fila < 3 and 0 <= nueva_col < 3:
                nuevo_tablero = deepcopy(self.tablero)
                nuevo_tablero[self.fila_vacia][self.columna_vacia] = nuevo_tablero[nueva_fila][nueva_col]
                nuevo_tablero[nueva_fila][nueva_col] = 0
                hijos.append(EstadoPuzzle(
                    nuevo_tablero, nueva_fila, nueva_col, self.nivel + 1, self))
        
        return hijos
    
    def __lt__(self, other):
        return self.costo_total < other.costo_total
    
    def __eq__(self, other):
        return self.tablero == other.tablero
    
    def __hash__(self):
        return hash(tuple(tuple(fila) for fila in self.tablero))
    
    def __str__(self):
        return '\n'.join([' '.join(map(str, fila)) for fila in self.tablero])
    
    def _repr_html_(self):
        """Representación HTML para mejor visualización en Jupyter"""
        html = "<table style='border-collapse: collapse; border: 2px solid black;'>"
        for fila in self.tablero:
            html += "<tr>"
            for valor in fila:
                color = "lightgray" if valor == 0 else "white"
                html += f"<td style='border: 1px solid black; width: 30px; height: 30px; text-align: center; background-color: {color}'>{valor if valor != 0 else ''}</td>"
            html += "</tr>"
        html += "</table>"
        return html

In [ ]:
def busqueda_anchura(estado_inicial):
    """Implementación de búsqueda en anchura"""
    from collections import deque
    cola = deque([estado_inicial])
    visitados = set()
    visitados.add(estado_inicial)
    
    while cola:
        estado_actual = cola.popleft()
        
        if estado_actual.es_estado_final():
            return reconstruir_camino(estado_actual)
        
        for hijo in estado_actual.generar_hijos():
            if hijo not in visitados:
                visitados.add(hijo)
                cola.append(hijo)
    
    return None

def busqueda_profundidad(estado_inicial, limite_profundidad=30):
    """Implementación de búsqueda en profundidad con límite"""
    pila = [estado_inicial]
    visitados = set()
    visitados.add(estado_inicial)
    
    while pila:
        estado_actual = pila.pop()
        
        if estado_actual.es_estado_final():
            return reconstruir_camino(estado_actual)
        
        if estado_actual.nivel < limite_profundidad:
            for hijo in estado_actual.generar_hijos():
                if hijo not in visitados:
                    visitados.add(hijo)
                    pila.append(hijo)
    
    return None

def busqueda_a_estrella(estado_inicial):
    """Implementación de búsqueda A* (heurística)"""
    cola_prioridad = []
    heapq.heappush(cola_prioridad, estado_inicial)
    visitados = set()
    visitados.add(estado_inicial)
    
    while cola_prioridad:
        estado_actual = heapq.heappop(cola_prioridad)
        
        if estado_actual.es_estado_final():
            return reconstruir_camino(estado_actual)
        
        for hijo in estado_actual.generar_hijos():
            if hijo not in visitados:
                visitados.add(hijo)
                heapq.heappush(cola_prioridad, hijo)
    
    return None

def reconstruir_camino(estado_final):
    """Reconstruye el camino desde el estado inicial hasta el final"""
    camino = []
    while estado_final:
        camino.append(estado_final)
        estado_final = estado_final.estado_anterior
    return list(reversed(camino))

In [ ]:
def mostrar_solucion(camino, nombre_algoritmo):
    """Muestra la solución encontrada por un algoritmo"""
    if camino is None:
        print(f"{nombre_algoritmo}: No se encontró solución.")
        return
    
    display(HTML(f"<h3>{nombre_algoritmo}: Solución encontrada en {len(camino)-1} movimientos</h3>"))
    
    # Mostrar solo algunos estados para no saturar la visualización
    pasos_mostrar = [0, len(camino)//2, len(camino)-1]  # Primer, medio y último paso
    
    for i, estado in enumerate(camino):
        if i in pasos_mostrar or i == 0 or i == len(camino)-1:
            display(HTML(f"<b>Movimiento {i}:</b>"))
            display(estado)
    
    # Mostrar resumen
    display(HTML(f"<p>Total de movimientos: {len(camino)-1}</p>"))

In [ ]:
# Tablero inicial
tablero_inicial = [
    [2, 8, 4],
    [7, 0, 1],
    [6, 5, 3]
]

# Crear estado inicial
estado_inicial = EstadoPuzzle(tablero_inicial, 1, 1)

# Ejecutar los algoritmos
display(HTML("<h2>Resolviendo 8-Puzzle con diferentes algoritmos</h2>"))

display(HTML("<h3>Estado inicial:</h3>"))
display(estado_inicial)

# Búsqueda en anchura
solucion_anchura = busqueda_anchura(estado_inicial)
mostrar_solucion(solucion_anchura, "Búsqueda en Anchura")

# Búsqueda en profundidad
solucion_profundidad = busqueda_profundidad(estado_inicial)
mostrar_solucion(solucion_profundidad, "Búsqueda en Profundidad")

# Búsqueda A*
solucion_a_estrella = busqueda_a_estrella(estado_inicial)
mostrar_solucion(solucion_a_estrella, "Búsqueda A* (Heurística)")